In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
def get_data(query):
    df=pd.read_sql(
    f'''
    {query}
    ''',
    con)
    return(df)

In [10]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [103]:
tasks+=1
pd.read_sql(
    ''' 
   SELECT COUNT (CRIM), COUNT (ZN), COUNT (INDUS), COUNT (CHAS), COUNT (NOX) FROM boston
   where CRIM is NULL or ZN is NULL or INDUS is NULL or CHAS is NULL or NOX is NULL

    ''',
    con,
    
)

,COUNT (CRIM),COUNT (ZN),COUNT (INDUS),COUNT (CHAS),COUNT (NOX)
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [110]:
tasks+=1
pd.read_sql(
    '''
    SELECT COUNT (distinct CRIM), 
    COUNT (distinct ZN),
      COUNT (distinct INDUS), 
      COUNT (distinct CHAS), 
      COUNT (distinct NOX) FROM boston  
    ''',
    con,
)

,COUNT (distinct CRIM),COUNT (distinct ZN),COUNT (distinct INDUS),COUNT (distinct CHAS),COUNT (distinct NOX)
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [386]:
tasks+=1
pd.read_sql(
    '''select column_name from (with a as(select CRIM as value , 'CRIM' as column_name, ROW_NUMBER() OVER(ORDER BY CRIM) ROW_N , min(CRIM) over() as min_val from boston union
select ZN as value , 'ZN' as column_name, ROW_NUMBER() OVER(ORDER BY ZN) ROW_N , min(ZN) over() as min_val from boston union
select INDUS as value , 'INDUS' as column_name, ROW_NUMBER() OVER(ORDER BY INDUS) ROW_N , min(INDUS) over() as min_val from boston union
select CHAS as value , 'CHAS' as column_name, ROW_NUMBER() OVER(ORDER BY CHAS) ROW_N , min(CHAS) over() as min_val from boston union
select NOX as value , 'NOX' as column_name, ROW_NUMBER() OVER(ORDER BY NOX) ROW_N , min(NOX) over() as min_val from boston)
select distinct column_name,min_val,avg(value) over(partition by column_name) as med from a 
where row_n in (253,254)) where min_val = med
    ''',
    con,
)

# больше половины домов не граничат с рекой 
# и не зонирована под участки 
# 

,column_name
0,CHAS
1,ZN


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [460]:
tasks+=1
pd.read_sql(
    '''
    select distinct criter,
    (select avg(rm) from boston where medv = (select max(medv) from boston)) avg_rm_max_medv,
    avg (rm) over(partition by criter) avg_rm_criter,
    (select avg(rm) from boston where medv = (select max(medv) from boston)) -avg (rm) over(partition by criter) dif_avg_rm 
    from (SELECT medv,rm, case
    when row_number() over(ORDER BY MEDV) <= 25 then 'bot_25'
    when row_number() over(ORDER BY MEDV) <= 50 then 'bot_50'
    when row_number() over(ORDER BY MEDV) <= 100 then 'bot_100'
    when row_number() over(ORDER BY MEDV) <= 200 then 'bot_200'
    when row_number() over(ORDER BY MEDV) <= 300 then 'bot_300'
    else "dorogo_bogato" end criter from boston)
    where criter !='dorogo_bogato'
    order by dif_avg_rm desc
    ''',
    con,
)

# влияние числа комнат очень значительное , 
#т.к разница между самым дорогоим сегментом и самым дешевым, всего 1.7 единицы

,criter,avg_rm_max_medv,avg_rm_criter,dif_avg_rm
0,bot_25,7.484,5.74784,1.73616
1,bot_50,7.484,5.75864,1.72536
2,bot_200,7.484,5.93629,1.54771
3,bot_100,7.484,6.02100,1.46300
4,bot_300,7.484,6.09327,1.39073


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [485]:
tasks+=1
pd.read_sql(
    '''select * from
    (select lstat,medv, rank() over (order by lstat) rank from boston)
    where medv = (select max(medv) from boston)
    ''',
    con,
)
# очевидно, что в самых дорогих домах процент насления с низким статус ниже,чем в других.
# Но нельзя сказать, что только в самых дорогих наименший процент такого населения]

,lstat,medv,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [168]:
tasks+=1
pd.read_sql(
    '''
    SELECT distinct avg(MEDV) OVER (PARTITION BY CHAS) as 'MEDV_avg',
    CASE 
    WHEN CHAS = 1 then 'Граничит_с_рекой' 
    ELSE 'Не_граничит_с_рекой' 
    END as CHAS_type
    FROM boston
  
    ''',
    con,
)

,MEDV_avg,CHAS_type
0,22.093843,Не_граничит_с_рекой
1,28.440000,Граничит_с_рекой


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [245]:
tasks+=1
pd.read_sql(
    '''
    select * from (SELECT ROUND(CRIM - avg(CRIM) OVER (),2) AS diff_CRIM,
    ROUND(ZN - avg(ZN) OVER (),2) AS diff_ZN,
    ROUND(INDUS- avg(INDUS) OVER (),2) AS diff_INDUS,
    ROUND(NOX- avg(NOX) OVER (),2) AS diff_NOX,
    CASE 
    WHEN CHAS = 1 then 'Граничит_с_рекой' 
    ELSE 'Не_граничит_с_рекой' 
    END as CHAS_type
    FROM boston)
    WHERE diff_CRIM > 0 OR diff_ZN > 0 OR diff_INDUS > 0 OR diff_NOX > 0


    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

До конца не уверен, как будет правильнее, думаю, что второй вариант более правильный

In [552]:
tasks+=1
pd.read_sql(
    ''' with table_1 as (SELECT nox, indus , ntile(10) over(order by nox,indus) as pers
    from boston)
     select avg(nox) avg_nox,avg(indus) avg_indus,pers from table_1 group by pers

    ''',
    con,
)
# есть явная корреляция между Indus и NOX

,avg_nox,avg_indus,pers
0,0.407692,3.637255,1
1,0.434153,6.533529,2
2,0.453443,5.596667,3
3,0.493157,6.255098,4
4,0.520118,7.680784,5
5,0.548471,9.933725,6
6,0.591440,18.418600,7
7,0.633540,17.182200,8
8,0.694060,18.100000,9
9,0.780300,18.573600,10


In [553]:
tasks+=1
pd.read_sql(
    ''' with table_1 as (SELECT nox, indus , ntile(10) over(order by nox) as pers_nox
    ,ntile(10) over(order by indus) as pers_indus
    from boston)
    select avg_nox, avg_indus, pers_nox as pers from  
    (select avg(nox) avg_nox ,pers_nox from table_1 group by pers_nox)
    join (select avg(indus) avg_indus ,pers_indus from table_1 group by pers_indus)
    on pers_nox = pers_indus

    ''',
    con,
)
# есть явная корреляция между Indus и NOX 

,avg_nox,avg_indus,pers
0,0.407692,2.016078,1
1,0.434153,3.645294,2
2,0.453443,5.246275,3
3,0.493157,6.471765,4
4,0.520118,8.256078,5
5,0.548471,10.749216,6
6,0.591440,17.245800,7
7,0.633540,18.100000,8
8,0.694060,18.307200,9
9,0.780300,21.938800,10


In [ ]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

In [264]:
[1,3,6,7,9,32,4,2,6,7,0,4,4,5,6,8,9,3,4,5,7,8,9,0]

[1, 3, 6, 7, 9, 32, 4, 2, 6, 7, 0, 4, 4, 5, 6, 8, 9, 3, 4, 5, 7, 8, 9, 0]

In [265]:
from statistics import median

In [266]:
a= [1,3,6,7,9,32,4,2,6,7,0,4,4,5,6,8,9,3,4,5,7,8,9,0]